## Trabajo 3 Grafos para Data Science - Parte Práctica (Código)
Integrantes: Álex Álvarez, Alfonso Tobar

### Carga de libraries a utilizar

In [14]:
# Importamos las libraries necesarias
import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.nn import GraphConv, SAGEConv, global_mean_pool
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader, Data
import numpy as np
import pandas as pd
import time

if not torch.cuda.is_available():
    print('Please Activate GPU Accelerator if available')
else:
    print('Everything is Set')

Everything is Set


### Implementación de los 5 modelos a utilizar

FFNN_1HL: red neuronal feed-forward con una capa escondida y función de activación ReLU.

In [15]:
class FFNN_1HL(nn.Module):
    def __init__(self, in_features, hidden_features, num_classes):
        super(FFNN_1HL, self).__init__()
        self.layer1 = nn.Linear(in_features, hidden_features)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_features, num_classes)

    def forward(self, data):
        x = data.x
        
        z1 = self.layer1(x)
        h1 = self.act1(z1)
        z2 = self.layer2(h1)
        return z2

GNN_1HL_GraphConv: red neuronal de grafos con una capa escondida, funciones de activación ReLU y suma como agregación sobre vecinos.

In [16]:
class GNN_1HL_GraphConv(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN_1HL_GraphConv, self).__init__()
        self.layer1 = GraphConv(input_dim, hidden_dim)
        self.act1 = nn.ReLU()
        self.layer2 = GraphConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        z1 = self.layer1(x, edge_index)
        h1 = self.act1(z1)
        z2 = self.layer2(h1, edge_index)
        return z2

GNN_2HL_GraphConv: red neuronal de grafos con dos capas escondidas, funciones de activación ReLU y suma como agregación sobre vecinos.

In [17]:
class GNN_2HL_GraphConv(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(GNN_2HL_GraphConv, self).__init__()
        self.layer1 = GraphConv(input_dim, hidden_dim1)
        self.act1 = nn.ReLU()
        self.layer2 = GraphConv(hidden_dim1, hidden_dim2)
        self.act2 = nn.ReLU()
        self.layer3 = GraphConv(hidden_dim2, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        z1 = self.layer1(x, edge_index)
        h1 = self.act1(z1)
        z2 = self.layer2(h1, edge_index)
        h2 = self.act2(z2)
        z3 = self.layer3(h2, edge_index)
        return z3

GNN_1HL_SAGEConv: red neuronal de grafos con una capa escondida, funciones de activación ReLU y media como agregación sobre vecinos.

In [18]:
class GNN_1HL_SAGEConv(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN_1HL_SAGEConv, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.act1 = nn.ReLU()
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        z1 = self.conv1(x, edge_index)
        h1 = self.act1(z1)
        z2 = self.conv2(h1, edge_index)
        return z2

GNN_2HL_SAGEConv: red neuronal de grafos con dos capas escondidas, funciones de activación ReLU y media como agregación sobre vecinos.

In [19]:
class GNN_2HL_SAGEConv(nn.Module):
    def __init__(self, in_channels, hidden_channels1, hidden_channels2, out_channels):
        super(GNN_2HL_SAGEConv, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels1)
        self.act1 = nn.ReLU()
        self.conv2 = SAGEConv(hidden_channels1, hidden_channels2)
        self.act2 = nn.ReLU()
        self.conv3 = SAGEConv(hidden_channels2, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        z1 = self.conv1(x, edge_index)
        h1 = self.act1(z1)
        z2 = self.conv2(h1, edge_index)
        h2 = self.act2(z2)
        z3 = self.conv3(h2, edge_index)
        return z3

### Función para evaluar los modelos

In [20]:
def evaluate_models(models, data, num_epochs = 10, num_trials = 5):
    # Inicializamos una lista para almacenar los resultados
    results = []
    # Iteramos num_trials veces sobre cada modelo a evaluar
    for trial in range(num_trials):
    # Iteramos sobre cada modelo a evaluar
        for model in models:
            start_time = time.time()
            # Inicializamos el optimizador Adam con los parámetros del modelo
            optimizer = torch.optim.Adam(model.parameters())
            # Inicializamos la función de pérdida como CrossEntropyLoss
            loss_function = torch.nn.CrossEntropyLoss()
            # Iteramos sobre las épocas de entrenamiento
            for epoch in range(num_epochs):
                # Calculamos la salida del modelo a partir del conjunto de entrenamiento
                out = model(data)
                # Calculamos la pérdida utilizando la función de pérdida
                loss = loss_function(out[data.train_mask], data.y[data.train_mask])
                # Realizamos el backpropagation para actualizar los parámetros del modelo
                loss.backward()
                optimizer.step()
                # Resetear los gradientes
                optimizer.zero_grad()

            # Calculamos la precisión del modelo en el conjunto de testeo
            _, pred = torch.max(model(data), dim=1)
            correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
            accuracy = correct / int(data.test_mask.sum())
            elapsed_time = time.time() - start_time
            # Guardamos resultados en tuplas (accuracy, elapsed_time)
            results.append((accuracy, elapsed_time))
    # Calculamos los promedios de accuracy por cada modelo
    avg_accuracy = [sum([result[0] for result in results[i::len(models)]])/num_trials for i in range(len(models))]
    # Calculamos los promedios de tiempo por cada modelo
    avg_time = [sum([result[1] for result in results[i::len(models)]])/num_trials for i in range(len(models))]
    # Retornamos los resultados
    return list(zip(avg_accuracy, avg_time))

### Función para setear datasets y número de nodos en los modelos:
0 para Cora, 1 para Citeseer, 2 para Pubmed.

In [21]:
def set_data_nodes(number = 0, batch_size= 32, shuffle=True, HLnodes = 100):
    # Nombres de los datasets
    datasets = ['Cora', 'Citeseer', 'Pubmed']
    # Importamos el dataset
    dataset = Planetoid(root='./data', name=datasets[number])
    data = dataset[0]
    # Guardamos el nombre del dataset seleccionado
    dataset_name = datasets[number]
    # Data loader (no es necesario para la implementación actual)
    data_loader = DataLoader(dataset[0], batch_size=batch_size, shuffle=shuffle)
    # Definimos los modelos con la cantidad de nodos en en cada layer
    models = [FFNN_1HL(dataset.num_node_features, HLnodes, dataset.num_classes),
                GNN_1HL_GraphConv(dataset.num_node_features, HLnodes, dataset.num_classes),
                GNN_2HL_GraphConv(dataset.num_node_features, HLnodes, HLnodes, dataset.num_classes),
                GNN_1HL_SAGEConv(dataset.num_node_features, HLnodes, dataset.num_classes),
                GNN_2HL_SAGEConv(dataset.num_node_features, HLnodes, HLnodes, dataset.num_classes)]
    # Retornamos los resultados

    return dataset, data_loader, data, dataset_name, models

### Setear dataset y nodos: (Usar para evaluar modelos en un único dataset)

In [22]:
# Elegimos el dataset 0, 1 o 2 para Cora, Citeseer o Pubmed respectivamente
# (Usar para evaluar modelos en un único dataset)
dataset, data_loader, data, dataset_name, models = set_data_nodes(0)

### Evaluar los modelos en el dataset escogido

In [23]:
# Evaluamos los modelos en uno de los datasets
# (Usar para evaluar modelos en un único dataset con cierto número de épocas e intentos)
results = evaluate_models(models, data, 1, 1)
# Imprimimos el nombre del dataset a evaluar
print("Dataset name: ", dataset_name)
# Imprimimos los promedios con el nombre de cada modelo
for i, model in enumerate(models):
    print(model.__class__.__name__,
            "Accuracy: {:.4f}".format(results[i][0]),
            "Time: {:.4f}".format(results[i][1]))

Dataset name:  Cora
FFNN_1HL Accuracy: 0.1270 Time: 0.0270
GNN_1HL_GraphConv Accuracy: 0.3040 Time: 0.0971
GNN_2HL_GraphConv Accuracy: 0.4630 Time: 0.1200
GNN_1HL_SAGEConv Accuracy: 0.1280 Time: 0.1031
GNN_2HL_SAGEConv Accuracy: 0.1810 Time: 0.1230


### Función para evaluar todos modelos en todos los dataset e imprimir sus resultados

In [24]:
# Función para evaluar todos modelos en todos los dataset e imprimir sus resultados promedio
def full_models_results(num_epochs = 10, num_trials = 5):
    # Evaluamos cada dataset
    for di in range(3):
        # Elegimos el dataset 0, 1 o 2 para Cora, Citeseer o Pubmed respectivamente
        dataset, data_loader, data, dataset_name, models = set_data_nodes(di)
        # Evaluamos los modelos
        results = evaluate_models(models, data, num_epochs, num_trials)
        # Imprimimos el dataset a evaluar
        print("Dataset name: ", dataset_name)
        # Imprimimos los promedios con el nombre de cada modelo
        for i, model in enumerate(models):
            print(model.__class__.__name__,
                    "Accuracy: {:.4f}".format(results[i][0]),
                    "Time: {:.4f}".format(results[i][1]))

In [25]:
# Imprimimos los resultados promedio de cada modelo para cada dataset
full_models_results()

Dataset name:  Cora
FFNN_1HL Accuracy: 0.4786 Time: 0.1733
GNN_1HL_GraphConv Accuracy: 0.7586 Time: 0.6252
GNN_2HL_GraphConv Accuracy: 0.7378 Time: 0.7243
GNN_1HL_SAGEConv Accuracy: 0.7698 Time: 0.6705
GNN_2HL_SAGEConv Accuracy: 0.7822 Time: 0.8001
Dataset name:  Citeseer
FFNN_1HL Accuracy: 0.5076 Time: 0.4645
GNN_1HL_GraphConv Accuracy: 0.6126 Time: 1.5416
GNN_2HL_GraphConv Accuracy: 0.5958 Time: 1.6608
GNN_1HL_SAGEConv Accuracy: 0.6564 Time: 1.6232
GNN_2HL_SAGEConv Accuracy: 0.6432 Time: 1.7810
Dataset name:  Pubmed
FFNN_1HL Accuracy: 0.4780 Time: 0.5642
GNN_1HL_GraphConv Accuracy: 0.7508 Time: 2.3166
GNN_2HL_GraphConv Accuracy: 0.7222 Time: 3.1818
GNN_1HL_SAGEConv Accuracy: 0.6718 Time: 2.4564
GNN_2HL_SAGEConv Accuracy: 0.7224 Time: 3.3296


## Discusión de Resultados

Respecto a los resultados podemos decir que, considerando los parámetros dados, las redes neuronales de grafos (GNN) son mejores en términos de accuracy que las redes neuronales feed-forward (FFNN) para resolver tareas en grafos como los datasets dados.

Además, se puede observar que aumentar el número de capas escondidas en las GNNs no siempre mejora el accuracy, y que el tipo de agregación utilizada también puede afectar el desempeño del modelo. También se puede ver que las GNNs tardan más en ejecutarse que las FFNNs debido a la complejidad adicional que implica el tratar con datos en forma de grafos y calcular agregaciones.